In [ ]:
%load_ext autoreload
%autoreload 2

# Basic Report

This notebook demonstrates a comprehensive foundation analysis using the rigid block methodology, following the Costa Rican Geotechnical Code (CGCR-2024).

## 1. Import Libraries

In [ ]:
# Standard Python libraries
import os
from datetime import datetime

# Third-party libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from ePy_analysis import Force, NodeReaction
from ePy_analysis.structure.foundations import PointFoundationObject
from ePy_analysis.structure.nodes import Node
from ePy_analysis.structure.soil import SoilType
from ePy_analysis.robot.nodes import RobotNodes, RobotSupports


from ePy_docs.reports.reporter import ReportWriter
from ePy_docs import ReadFiles
from ePy_docs.styler.colors import TableColorConfig

from ePy_docs.project.setup import DirectoryConfig, DirectoryConfigSettings, ProjectFolders,clear_current_project_config

from ePy_docs.units.units import auto_detect_and_convert_units
from ePy_docs.units.converter import UnitConverter, get_unit_from_config


## 2. Project Setup

### Initialize project configuration

In [ ]:
# CONFIGURACIÓN DEL PROYECTO - EJECUTAR PRIMERO
import os
from ePy_docs.project.setup import DirectoryConfig, DirectoryConfigSettings

current_notebook_dir = os.getcwd()

# Create settings object with JSON sync enabled
settings = DirectoryConfigSettings(
    base_dir=current_notebook_dir,
    json_templates=True,
    auto_create_dirs=True
)

# Create project configuration
project_config = DirectoryConfig.from_settings(settings)

# Setup the project with JSON synchronization
project_config.setup_project(sync_json=True)

In [ ]:
print(f"Project directory: {project_config.base_dir}")

### Load all parameters
Se cargan los parámetros en variables de entorno para su uso posterior.

```{r}


In [ ]:
configs = project_config.load_all_configs()
units_config = configs['units']
project_config_data = configs['project']
color_config = TableColorConfig(
    palette=project_config_data.get('styling', {}).get('table_palette', 'viridis'),
    center_value=project_config_data.get('styling', {}).get('center_value', 50),
    reverse=project_config_data.get('styling', {}).get('reverse', True),
    header_color=project_config_data.get('styling', {}).get('header_color', 'lightblue'),
)

### Load analysis data
Se cargan los datos de análisis necesarios para la evaluación de la cimentación.

```{r}

In [ ]:
# File paths from refreshed configuration
blocks_csv_path = project_config.files.blocks_csv
nodes_csv_path = project_config.files.nodes_csv
reactions_csv_path = project_config.files.reactions_csv

# Debug: Print the actual paths being used
print(f"Blocks CSV: {blocks_csv_path}")
print(f"Nodes CSV: {nodes_csv_path}")
print(f"Reactions CSV: {reactions_csv_path}")

# Verify files exist
import os
files_status = {
    'blocks_csv': os.path.exists(blocks_csv_path),
    'nodes_csv': os.path.exists(nodes_csv_path),
    'reactions_csv': os.path.exists(reactions_csv_path)
}

print(f"Files exist: {files_status}")

if not all(files_status.values()):
    missing = [k for k, v in files_status.items() if not v]
    raise FileNotFoundError(f"Data files not found: {missing}")

In [ ]:
# Force clean restart of configuration to pick up setup.json changes

# Clear any cached configuration
clear_current_project_config()

# Recreate the configuration from scratch
settings = DirectoryConfigSettings(
    base_dir=current_notebook_dir,
    json_templates=True,
    auto_create_dirs=True
)

project_config = DirectoryConfig.from_settings(settings)
project_config.setup_project(sync_json=True)

# Validate configuration with fresh settings
validation = project_config.validate_required_files()
missing_files = [k for k, v in validation.items() if not v]
if missing_files:
    raise FileNotFoundError(f"Missing required files: {missing_files}")

### Define job units and init unit converter

Las unidades de trabajo que se espera utilizar en este proyecto son:

In [ ]:
# Units from configuration only
length_unit = get_unit_from_config(units_config, 'structure_dimensions', 'length')
area_unit = get_unit_from_config(units_config, 'structure_dimensions', 'structure_area')
volume_unit = get_unit_from_config(units_config, 'section_dimensions', 'length3')
force_unit = get_unit_from_config(units_config, 'forces', 'force')
moment_unit = get_unit_from_config(units_config, 'forces', 'moment')
converter = UnitConverter.create_default()

Cuando se importe una base de datos de secciones u otros elementos, el sistema automáticamente ajustará la información a las unidades establecidas.

### Define report parameters

El reporte será ubicado en el folder que se establezca para este fin

In [ ]:
# Report configuration from project settings
reports_dir = project_config.folders.results
report_filename = project_config_data.get('output', {}).get('report_filename', 'foundation_analysis_report.pdf')
report_path = os.path.join(reports_dir, report_filename)

writer = ReportWriter(
    file_path=report_path,
    auto_print=project_config_data.get('output', {}).get('auto_print', True)
)


Se inicializan las clases que se utilizarán para la generación del reporte.

```{r}

In [ ]:
# Initialize data processing classes
nodes = RobotNodes(filepath=nodes_csv_path)
nodes_df = nodes.import_coordinates()
support_nodes_df = nodes.get_supports_info()

robot_reactions = RobotSupports(filepath=reactions_csv_path)
reactions_df = robot_reactions.import_reactions()


## 3. Report

In [ ]:
report_config = project_config_data.get('report', {})
writer.add_responsability_page()


In [ ]:
writer.add_h1(report_config.get('title', 'Foundation Analysis Report'))

writer.add_tip(
    f"""📏 Length unit: ${length_unit}$
    📐 Area unit: ${area_unit}$
    📦 Volume unit: ${volume_unit}$
    🏋️ Force unit: ${force_unit}$
    ⚖️ Moment unit: ${moment_unit}$
""", "prueba con emojis")

writer.add_tip(
    f"""📏 Length unit: ${length_unit}$
    📐 Area unit: ${area_unit}$
    📦 Volume unit: ${volume_unit}$
    🏋️ Force unit: ${force_unit}$
    ⚖️ Moment unit: ${moment_unit}$
""", "prueba con emojis")

writer.add_h1(report_config.get('title', 'Foundation Analysis Report'))

writer.add_tip(
    f"""Length unit: ${length_unit}$
    Area unit: ${area_unit}$
    Volume unit: ${volume_unit}$
    Force unit: ${force_unit}$
    Moment unit: ${moment_unit}$
""", "prueba sin emojis")

writer.add_h1(report_config.get('title', 'Foundation Analysis Report'))

writer.add_warning(
    f"""Length unit: ${length_unit}$
Area unit: ${area_unit}$
Volume unit: ${volume_unit}$
Force unit: ${force_unit}$
Moment unit: ${moment_unit}$
""", "prueba sin emojis ni espacios")

writer.add_tip(
    report_config.get('warning_text', 'Generated report - verify all calculations'),
    report_config.get('warning_title', 'Important Notice')
    )


writer.add_h1(report_config.get('title', 'Foundation Analysis Report'))

writer.add_success(
    f"""Length unit: $${length_unit}$$
Area unit: $${area_unit}$$
Volume unit: $${volume_unit}$$
Force unit: $${force_unit}$$
Moment unit: $${moment_unit}$$
""", "prueba sin emojis ni espacios")

writer.add_warning(
    report_config.get('warning_text', 'Generated report - verify all calculations'),
    report_config.get('warning_title', 'Important Notice')
    )

# 📋 AGREGAR CONTENIDO ADICIONAL AL INFORME
# Aprovechando las capacidades de ePy_suite para enriquecer el PDF

# 1. Agregar análisis de conversión de unidades
writer.add_h2("Unit Conversion Analysis")
writer.add_content("""
This section demonstrates the unit conversion capabilities integrated within the project. 
The ePy_suite system provides comprehensive support for engineering unit conversions, 
which is crucial for international projects and multi-system calculations.
""")

# Crear datos de ejemplo para conversiones de unidades
from ePy_docs.units.converter import UnitConverter

# Usar el converter ya inicializado
force_conversions = []
moment_conversions = []
stress_conversions = []

# Ejemplos de conversiones de fuerzas
force_examples = [
    (1000, 'kgf', 'kN'),
    (500, 'kgf', 'lbf'),
    (10, 'kN', 'tonf'),
    (2500, 'lbf', 'kgf'),
    (100, 'ksi', 'kgf/cm2'),
    (1, 'tonf', 'kN'),
    (2500, 'kgf/cm2', 'psi'),
    (7030.70, 'kgf/cm2', 'ksi'),
    (703, 'MPa', 'ksi'),
    (70, 'MPa', 'kgf/cm2'),
    (1000, 'N', 'kN'),
    (500, 'N', 'lbf'),
    (25, 'N', 'dyn'),
    (10000, 'Pa', 'kgf/m2'),
    (1, 'atm', 'bar'),
    (2.5, 'bar', 'psi'),
    (600, 'psi', 'MPa'),
    (200, 'kgf', 'N'),
    (50, 'kgf', 'dyn'),
    (3500, 'N·m', 'kgf·m'),
    (500, 'N·m', 'lbf·ft'),
    (703, 'MPa', 'bar'),
]

for value, from_unit, to_unit in force_examples:
    try:
        result = converter.universal_unit_converter(value, from_unit, to_unit)
        if isinstance(result, (int, float)):
            force_conversions.append({
                'Original Value': f"{value} {from_unit}",
                'Converted Value': f"{result:.2f} {to_unit}",
                'Conversion Factor': f"1 {from_unit} = {result/value:.6f} {to_unit}"
            })
        else:
            force_conversions.append({
                'Original Value': f"{value} {from_unit}",
                'Converted Value': f"{result} {to_unit}",
                'Conversion Factor': "See conversion"
            })
    except Exception as e:
        force_conversions.append({
            'Original Value': f"{value} {from_unit}",
            'Converted Value': "Conversion Error",
            'Conversion Factor': f"Error: {str(e)}"
        })

# Crear DataFrame y agregar al reporte
if force_conversions:
    force_df = pd.DataFrame(force_conversions)
    writer.add_h3("Force Unit Conversions")
    writer.add_content("Examples of force unit conversions commonly used in structural analysis:")
    writer.add_table(force_df, title="Force unit conversion examples")

writer.add_h2("Design Recommendations and Best Practices")
writer.add_content("""
Based on the analysis performed, the following recommendations are provided:

### Structural Design Recommendations:
1. **Foundation Sizing**: All calculated foundations should be reviewed for constructability constraints
2. **Load Distribution**: Consider load redistribution for heavily loaded nodes
3. **Construction Tolerances**: Include appropriate tolerances in final dimensions
4. **Material Specifications**: Ensure concrete strength meets or exceeds design requirements

### Quality Control Measures:
                   
$$F = ma$$ {#eq-newton2}
                   
$M + 2 = 0$
                   
1. **Dimensional Verification**: Verify (@eq-newton2) all foundation dimensions before construction
2. **Reinforcement Detailing**: Ensure proper reinforcement placement and cover
3. **Concrete Quality**: Implement proper concrete curing and quality control procedures
4. **Load Testing**: Consider load testing for critical [@CSCR2010_14] foundations

### Maintenance Considerations:
1. **Inspection Schedule**: Establish regular inspection intervals
2. **Settlement Monitoring**: Monitor foundation settlements over time
3. **Drainage Maintenance**: Ensure proper drainage around foundations
4. **Documentation**: Maintain as-built drawings and inspection records
""")

# 4. Agregar información técnica adicional usando las capacidades de conversión
writer.add_h2("Technical Specifications and Conversions")

# 5. Agregar análisis de comparación de métodos
writer.add_h2("Methodology Comparison")
writer.add_warning("""
                   
### Foundation Analysis Methods Comparison

The rigid block method used in this @tbl-6 analysis offers several advantages:

**Advantages:**
- Simple and conservative approach
- Well-established in engineering practice
- Suitable for preliminary design phases
- Compatible with various soil types

**Limitations:**
- May be overly conservative for some applications
- Does not account for soil-structure interaction effects
- Limited consideration of dynamic loading conditions

**Alternative Methods:**
- Finite Element Analysis (FEA) for complex geometries
- Spring-based models for dynamic analysis
- Advanced bearing capacity theories for special conditions
""")


### Nodes and supports

In [ ]:
writer.add_h2(report_config.get('nodes_section_title', 'Nodes and Supports Data'))
writer.add_content(report_config.get('nodes_description', 'Node coordinates and support information:'))

### Add "elegant" table with nodes coordinates

In [ ]:
writer.add_table(
    nodes_df, 
    title=report_config.get('nodes_table_title', 'Node coordinates'),
    max_rows_per_table=report_config.get('max_rows_per_table', [20, 20, 20])
)

### Add "elegant" table with supports coordinates

In [ ]:
writer.add_content(report_config.get('supports_description', 'Support information:'))
writer.add_table(
    support_nodes_df, 
    title=report_config.get('supports_table_title', 'Support nodes')
)

### Add Colored Table with Nodes Coordinates

In [ ]:
writer.add_colored_table(
    support_nodes_df, 
    title=report_config.get('supports_colored_table_title', 'Support nodes with reactions'),
    palette_name=report_config.get('table_palette', 'engineering'),
    highlight_columns=[f'X ({length_unit})', f'Z ({length_unit})', 'Support']
)

### Table subsets

#### Subset of columns

In [ ]:
writer.add_colored_table(
    nodes_df.head(report_config.get('sample_size', 5)), 
    title=report_config.get('sample_table_title', 'Sample node coordinates'),
    highlight_columns=[f'X ({length_unit})', f'Y ({length_unit})', f'Z ({length_unit})', 'Support']
)

#### Subset of rows

In [ ]:
writer.add_colored_table(
    nodes_df.head(report_config.get('sample_size', 5)),
    title=report_config.get('filtered_sample_title', 'Filtered sample coordinates'),
    highlight_columns=[f'Y ({length_unit})', f'X ({length_unit})'], 
    hide_columns=report_config.get('hidden_columns', [f'Z ({length_unit})', 'support'])
)

#### Subset of rows + columns

In [ ]:
writer.add_colored_table(
    nodes_df,
    title=report_config.get('subset_table_title', 'Subset coordinates'),
    highlight_columns=[f'Y ({length_unit})', f'X ({length_unit})'],
    hide_columns=report_config.get('hidden_columns', [f'Z ({length_unit})', 'support']),
    filter_by=('Node', report_config.get('filter_nodes', ['1', '2', '3']))
)

## Sort by

In [ ]:
reactions_df

In [ ]:
writer.add_colored_table(
    reactions_df, 
    title=report_config.get('reactions_table_title', 'Support reactions'),
    palette_name=report_config.get('table_palette', 'engineering'),
    highlight_columns=[f'FX ({force_unit})', f'FZ ({force_unit})', f'MX ({moment_unit})'],
    sort_by=(f'FZ ({force_unit})', report_config.get('sort_order', 'desc')),
    n_rows=5
)

In [ ]:
writer.add_colored_table(
    reactions_df.head(report_config.get('max_reaction_rows', 5)),
    title=report_config.get('reactions_summary_title', 'Reactions summary'),
    highlight_columns=[f'FX ({force_unit})', f'FZ ({force_unit})', f'MY ({moment_unit})']
)

writer.add_h2(report_config.get('blocks_section_title', 'Block Sizes Analysis'))

# Soil properties

# Sulzberger

## 5. Enhanced DCR Analysis with Unit Conversion

In [ ]:
# 📋 AGREGAR CONTENIDO ADICIONAL AL INFORME
# Aprovechando las capacidades de ePy_suite para enriquecer el PDF

# 1. Agregar análisis de conversión de unidades
writer.add_h2("Unit Conversion Analysis")
writer.add_content("""
This section demonstrates the unit conversion capabilities integrated within the project. 
The ePy_suite system provides comprehensive support for engineering unit conversions, 
which is crucial for international projects and multi-system calculations.
""")

# Crear datos de ejemplo para conversiones de unidades
from ePy_docs.units.converter import UnitConverter

# Usar el converter ya inicializado
force_conversions = []
moment_conversions = []
stress_conversions = []

# Ejemplos de conversiones de fuerzas
force_examples = [
    (1000, 'kgf', 'kN'),
    (500, 'kgf', 'lbf'),
    (10, 'kN', 'tonf'),
    (2500, 'lbf', 'kgf'),
    (100, 'ksi', 'kgf/cm2'),
    (1, 'tonf', 'kN'),
    (2500, 'kgf/cm2', 'psi'),
    (7030.70, 'kgf/cm2', 'ksi'),
    (703, 'MPa', 'ksi'),
    (70, 'MPa', 'kgf/cm2'),
    (1000, 'N', 'kN'),
    (500, 'N', 'lbf'),
    (25, 'N', 'dyn'),
    (10000, 'Pa', 'kgf/m2'),
    (1, 'atm', 'bar'),
    (2.5, 'bar', 'psi'),
    (600, 'psi', 'MPa'),
    (200, 'kgf', 'N'),
    (50, 'kgf', 'dyn'),
    (3500, 'N·m', 'kgf·m'),
    (500, 'N·m', 'lbf·ft'),
    (703, 'MPa', 'bar'),
    (1000, 'N', 'kN'),
]

for value, from_unit, to_unit in force_examples:
    try:
        result = converter.universal_unit_converter(value, from_unit, to_unit)
        if isinstance(result, (int, float)):
            force_conversions.append({
                'Original Value': f"{value} {from_unit}",
                'Converted Value': f"{result:.2f} {to_unit}",
                'Conversion Factor': f"1 {from_unit} = {result/value:.6f} {to_unit}"
            })
        else:
            force_conversions.append({
                'Original Value': f"{value} {from_unit}",
                'Converted Value': f"{result} {to_unit}",
                'Conversion Factor': "See conversion"
            })
    except Exception as e:
        force_conversions.append({
            'Original Value': f"{value} {from_unit}",
            'Converted Value': "Conversion Error",
            'Conversion Factor': f"Error: {str(e)}"
        })

# Crear DataFrame y agregar al reporte
if force_conversions:
    force_df = pd.DataFrame(force_conversions)
    writer.add_h3("Force Unit Conversions")
    writer.add_content("Examples of force unit conversions commonly used in structural analysis:")
    writer.add_table(force_df, title="Force unit conversion examples")

writer.add_h2("Design Recommendations and Best Practices")
writer.add_content("""
Based on the analysis performed, the following recommendations are provided:

### Structural Design Recommendations:
1. **Foundation Sizing**: All calculated foundations should be reviewed for constructability constraints
2. **Load Distribution**: Consider load redistribution for heavily loaded nodes
3. **Construction Tolerances**: Include appropriate tolerances in final dimensions
4. **Material Specifications**: Ensure concrete strength meets or exceeds design requirements

### Quality Control Measures:
                   
$$
F = m \cdot a
$$ {#eq-newton}
                   
$M + 2 = 0$
                   
1. **Dimensional Verification**: Verify (@eq-newton) all foundation dimensions before construction
2. **Reinforcement Detailing**: Ensure proper reinforcement placement and cover
3. **Concrete Quality**: Implement proper concrete curing and quality control procedures
4. **Load Testing**: Consider load testing for critical [@CSCR2010_14] foundations

### Maintenance Considerations:
1. **Inspection Schedule**: Establish regular inspection intervals
2. **Settlement Monitoring**: Monitor foundation settlements over time
3. **Drainage Maintenance**: Ensure proper drainage around foundations
4. **Documentation**: Maintain as-built drawings and inspection records
""")

# 4. Agregar información técnica adicional usando las capacidades de conversión
writer.add_h2("Technical Specifications and Conversions")

# 5. Agregar análisis de comparación de métodos
writer.add_h2("Methodology Comparison")
writer.add_warning("""
                   
### Foundation Analysis Methods Comparison

The rigid block method used in this @tbl-6 analysis offers several advantages:

**Advantages:**
- Simple and conservative approach
- Well-established in engineering practice
- Suitable for preliminary design phases
- Compatible with various soil types

**Limitations:**
- May be overly conservative for some applications
- Does not account for soil-structure interaction effects
- Limited consideration of dynamic loading conditions

**Alternative Methods:**
- Finite Element Analysis (FEA) for complex geometries
- Spring-based models for dynamic analysis
- Advanced bearing capacity theories for special conditions
""")

writer.generate(pdf=True, markdown=True, html=True)

In [ ]:
writer.add_tip(""" 

## Prueba 
                
* This report was generated using ePy Suite, a Python-based engineering analysis and design tool. 
* ePy Suite is designed to facilitate structural analysis, foundation design, and other engineering tasks.
* It provides a comprehensive framework for managing engineering projects, including unit conversions, data handling, and For more information, visit [ePy Suite Documentation](https://epy-suite.readthedocs.io).
                
### Prueba 
                
* This report was generated using ePy Suite, a Python-based engineering analysis and design tool.
* ePy Suite is designed to facilitate structural analysis, foundation design, and other engineering tasks.
* It provides a comprehensive framework for managing engineering projects, including unit conversions, data handling, and
For more information, visit [ePy Suite Documentation](https://epy-suite.readthedocs.io).

## Prueba 
                
* This report was generated using ePy Suite, a Python-based engineering analysis and design tool.
* ePy Suite is designed to facilitate structural analysis, foundation design, and other engineering tasks.
* It provides a comprehensive framework for managing engineering projects, including unit conversions, data handling, and
For more information, visit [ePy Suite Documentation](https://epy-suite.readthedocs.io).

### Prueba 
                
* This report was generated using ePy Suite, a Python-based engineering analysis and design tool.
* ePy Suite is designed to facilitate structural analysis, foundation design, and other engineering tasks.
* It provides a comprehensive framework for managing engineering projects, including unit conversions, data handling, and
For more information, visit [ePy Suite Documentation](https://epy-suite.readthedocs.io).   

### Prueba 
                
* This report was generated using ePy Suite, a Python-based engineering analysis and design tool.
* ePy Suite is designed to facilitate structural analysis, foundation design, and other engineering tasks.
* It provides a comprehensive framework for managing engineering projects, including unit conversions, data handling, and
For more information, visit [ePy Suite Documentation](https://epy-suite.readthedocs.io).
                
#### Prueba 
                
* This report was generated using ePy Suite, a Python-based engineering analysis and design tool.
* ePy Suite is designed to facilitate structural analysis, foundation design, and other engineering tasks.
* It provides a comprehensive framework for managing engineering projects, including unit conversions, data handling, and
For more information, visit [ePy Suite Documentation](https://epy-suite.readthedocs.io).

## Prueba 
                
* This report was generated using ePy Suite, a Python-based engineering analysis and design tool.
* ePy Suite is designed to facilitate structural analysis, foundation design, and other engineering tasks.
* It provides a comprehensive framework for managing engineering projects, including unit conversions, data handling, and
For more information, visit [ePy Suite Documentation](https://epy-suite.readthedocs.io).
       
""", title="Prueba final")

writer.add_tip(""" 

## Prueba 
                
This report was generated using ePy Suite, a Python-based engineering analysis and design tool.
ePy Suite is designed to facilitate structural analysis, foundation design, and other engineering tasks.
It provides a comprehensive framework for managing engineering projects, including unit conversions, data handling, and
For more information, visit [ePy Suite Documentation](https://epy-suite.readthedocs.io).

## Prueba 
                
* This report was generated using ePy Suite, a **Python-based** engineering analysis and design tool.
* ePy Suite is designed to facilitate structural analysis, foundation design, and other engineering tasks.
* It provides a comprehensive framework for managing engineering projects, including unit conversions, data handling, and
For more information, visit [ePy Suite Documentation](https://epy-suite.readthedocs.io).
                   
### Prueba 
                
* This report was generated using ePy Suite, a **Python-based** engineering analysis and design tool.
* ePy Suite is designed to facilitate structural analysis, foundation design, and other engineering tasks.
* It provides a comprehensive framework for managing engineering projects, including unit conversions, data handling, and
For more information, visit [ePy Suite Documentation](https://epy-suite.readthedocs.io).

#### Prueba 
                
* This report was generated using ePy Suite, a **Python-based** engineering analysis and design tool.
* ePy Suite is designed to facilitate structural analysis, foundation design, and other engineering tasks.
* It provides a comprehensive framework for managing engineering projects, including unit conversions, data handling, and
For more information, visit [ePy Suite Documentation](https://epy-suite.readthedocs.io).
                   
""")


writer.generate(pdf=True, html=True, citation_style='apa')

In [ ]:
writer.generate(qmd=True, pdf=True, html=True, tex=True, markdown=True, citation_style="ieee")